# 3A.mr - Reservoir Sampling distribué - énoncé - correction

Correction.

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## création d'un fichier à sampler

In [2]:
with open("sample4.txt", "w", encoding="utf8") as f:
    for i in range(0,100000):
        f.write("{0}\t{1}{0}\n".format(i, chr(i%26 + 65)))
    f.write("100001\tAAAAAA")

In [3]:
%load_ext pyensae
%head sample4.txt

## connexion

In [4]:
import os
blobhp = {}
if "HDCREDENTIALS" in os.environ:
    blobhp["blob_storage"], blobhp["password1"], blobhp["hadoop_server"], blobhp["password2"], blobhp["username"] = \
        os.environ["HDCREDENTIALS"].split("**")
    r = type(blobhp)
else:
    from pyquickhelper.ipythonhelper import open_html_form
    params={"blob_storage":"", "password1":"", "hadoop_server":"", "password2":"", "username":"axavier"}
    r = open_html_form(params=params,title="server + hadoop + credentials", key_save="blobhp")
r

dict

In [5]:
import pyensae
%load_ext pyensae
%load_ext pyenbc
blobstorage = blobhp["blob_storage"]
blobpassword = blobhp["password1"]
hadoop_server = blobhp["hadoop_server"]
hadoop_password = blobhp["password2"]
username = blobhp["username"]

In [6]:
client, bs = %hd_open
client, bs

(<pyensae.remote.azure_connection.AzureClient at 0x942e860>,
 <azure.storage.blob.blobservice.BlobService at 0x942e898>)

## upload du fichier

In [7]:
%blob_up sample3.txt /$PSEUDO/sampling/sample4.txt

'$PSEUDO/sampling/sample4.txt'

In [8]:
%blob_ls /$PSEUDO/sampling

,name,last_modified,content_type,content_length,blob_type
0,axavier/sampling/datafu-1.2.0.jar,"Fri, 13 Nov 2015 00:03:49 GMT",application/octet-stream,1600826,BlockBlob
1,axavier/sampling/out_sampled_rs4_2015.txt,"Fri, 13 Nov 2015 01:08:22 GMT",,0,BlockBlob
2,axavier/sampling/out_sampled_rs4_2015.txt/_SUC...,"Fri, 13 Nov 2015 01:08:22 GMT",application/octet-stream,0,BlockBlob
3,axavier/sampling/out_sampled_rs4_2015.txt/part...,"Fri, 13 Nov 2015 01:08:21 GMT",application/octet-stream,12785,BlockBlob
4,axavier/sampling/sample.txt,"Fri, 13 Nov 2015 00:02:50 GMT",application/octet-stream,1377780,BlockBlob
5,axavier/sampling/sample2.txt,"Fri, 13 Nov 2015 00:35:55 GMT",application/octet-stream,1377793,BlockBlob
6,axavier/sampling/sample3.txt,"Fri, 13 Nov 2015 00:39:40 GMT",application/octet-stream,1377793,BlockBlob
7,axavier/sampling/sample4.txt,"Sun, 15 Nov 2015 12:24:22 GMT",application/octet-stream,1377793,BlockBlob
8,axavier/sampling/sample4_hash.txt,"Fri, 13 Nov 2015 14:50:39 GMT",,0,BlockBlob
9,axavier/sampling/sample4_hash.txt/_SUCCESS,"Fri, 13 Nov 2015 14:50:39 GMT",application/octet-stream,0,BlockBlob


## Code python pour le reservoir sampling

In [9]:
ensemble = [ "%d%s" % (i, chr(i%26 + 97)) for i in range(0,10000)]
ensemble[:5]

['0a', '1b', '2c', '3d', '4e']

In [10]:
import random
def reservoir_sampling(ensemble, k):
    N = len(ensemble)
    echantillon = []
    for i, e in enumerate(ensemble):
        if len(echantillon) < k:
            echantillon.append(e)
        else:
            j = random.randint(0, i)
            if j < k:
                echantillon[j] = e
    return echantillon

reservoir_sampling(ensemble, 10)

['8681x',
 '8356k',
 '5490e',
 '4405l',
 '5890o',
 '2689l',
 '8672o',
 '3603p',
 '8599t',
 '6086c']

## python à jython

On s'assure que le code précédent fonctionne en jython (python compilé en java). On s'inspire pour cela de la documentation [jython-udfs](https://pig.apache.org/docs/r0.12.0/udf.html#jython-udfs).

### On créé d'abord un script PIG pour récupérer le schema et les premières lignes

In [11]:
%%PIG sample_explore.pig

ensemble = LOAD '$CONTAINER/$PSEUDO/sampling/sample4.txt' 
        USING PigStorage('\t') AS (x:int, v:chararray) ;
DESCRIBE ensemble;
ens_group = GROUP ensemble ALL;
DESCRIBE ens_group;
sampled = FOREACH ens_group GENERATE FLATTEN(ensemble);
DESCRIBE sampled;

--ens10 = LIMIT ensemble 10;
--ens_group10 = LIMIT en_group10 ;
--DUMP ens10;
--DUMP ens_group10;

Si la fonction suivante provoque une erreur ::

    AzureException: STATUS: 403, JSON: Expecting value: line 1 column 1 (char 0)
    <Response [403]>
    unable to submit job: sample_explore.pig
    
Vérifier les identifiants utilisés pour se connecter.

In [12]:
jid = %hd_pig_submit sample_explore.pig
jid

{'id': 'job_1446540516812_0185'}

In [13]:
st = %hd_job_status jid["id"]
(st["id"],st["percentComplete"],st["completed"],
st["status"]["jobComplete"],st["status"]["state"])

('job_1446540516812_0185', None, None, False, 'RUNNING')

La sortie standard contient les informations souhaitées :

In [14]:
%hd_tail_stderr jid["id"] -n 5

Et la sortie du second dump :: 
    
    (all,{(100001,AAAAAA),(99999,D99999),(99998,C99998)...

### Le code Jython

In [15]:
import pyensae

In [16]:
%%PYTHON reservoir_sampling.py

import random

@outputSchemaFunction("rsSchema")
def reservoir_sampling(ensemble):
    ensemble = eval(ensemble)
    k = 10
    N = len(ensemble)
    echantillon = []
    for i, e in enumerate(ensemble):
        if len(echantillon) < k:
            echantillon.append(e)
        else:
            j = random.randint(0, i)
            if j < k:
                echantillon[j] = e
    return echantillon

@schemaFunction("rsSchema")
def rsSchema(input):
    return input

In [17]:
%%jython reservoir_sampling.py reservoir_sampling
{(100001,"AAAAAA"),(99999,"D99999"),(99998,"C99998")}

On ajoute le code jython au script précédent :

In [18]:
%%PIG sample_explore_complete.pig

REGISTER '$CONTAINER/$SCRIPTPIG/reservoir_sampling.py' using jython as myrs;

ensemble = LOAD '$CONTAINER/$PSEUDO/sampling/sample4.txt' 
        USING PigStorage('\t') AS (x:int, v:chararray) ;
DESCRIBE ensemble;
ens_group = GROUP ensemble ALL;
DESCRIBE ens_group;
sampled = FOREACH ens_group GENERATE FLATTEN(myrs.reservoir_sample(ensemble));
DESCRIBE sampled;

STORE sampled INTO 
INTO '$CONTAINER/$PSEUDO/sampling/sample_rs.txt' USING PigStorage();

In [19]:
jid = %hd_pig_submit sample_explore_complete.pig -d reservoir_sampling.py
jid

{'id': 'job_1446540516812_0229'}

In [20]:
st = %hd_job_status jid["id"]
(st["id"],st["percentComplete"],st["completed"],
st["status"]["jobComplete"],st["status"]["state"])

('job_1446540516812_0229', None, 'done', False, 'RUNNING')

In [21]:
%hd_tail_stderr jid["id"] -n 100

A corriger plus tard. Dans l'immédiat, on utilisera la librairie [datafu](https://datafu.incubator.apache.org/docs/datafu/guide/sampling.html). Si le cluster ne reconnaît pas la librairie, voir la section java pour comprendre comment l'importer. On la déclare dans le script par l'instruction ``REGISTER``.

In [22]:
%%PIG sample_explore_datafu.pig

REGISTER '$CONTAINER/$PSEUDO/sampling/datafu-1.2.0.jar';
DEFINE RS datafu.pig.sampling.ReservoirSample('1000');

ensemble = LOAD '$CONTAINER/$PSEUDO/sampling/sample4.txt' 
        USING PigStorage('\t') AS (x:int, v:chararray) ;
DESCRIBE ensemble;
ens_group = GROUP ensemble ALL;
DESCRIBE ens_group;
sampled = FOREACH ens_group GENERATE FLATTEN(RS(ensemble));
DESCRIBE sampled;

STORE sampled 
INTO '$CONTAINER/$PSEUDO/sampling/sample_datafu_rs.txt' USING PigStorage();

In [23]:
jid = %hd_pig_submit sample_explore_datafu.pig
jid

{'id': 'job_1446540516812_0193'}

In [24]:
st = %hd_job_status jid["id"]
(st["id"],st["percentComplete"],st["completed"],
st["status"]["jobComplete"],st["status"]["state"])

('job_1446540516812_0193', '50% complete', None, False, 'RUNNING')

In [25]:
%hd_tail_stderr jid["id"] -n 100

In [26]:
%blob_ls /$PSEUDO/sampling/sample_datafu

,name,last_modified,content_type,content_length,blob_type
0,axavier/sampling/sample_datafu_rs.txt,"Sun, 15 Nov 2015 13:23:40 GMT",,0,BlockBlob
1,axavier/sampling/sample_datafu_rs.txt/_SUCCESS,"Sun, 15 Nov 2015 13:23:40 GMT",application/octet-stream,0,BlockBlob
2,axavier/sampling/sample_datafu_rs.txt/part-r-0...,"Sun, 15 Nov 2015 13:23:38 GMT",application/octet-stream,12780,BlockBlob


## version distribuée

Astuce : on distribue puis on recombine les échantillons en faisant un dernier reservoir sampling mais pondéré. Comment distribuer ? Le second sampling est remplacé par une méthode d'échantillonage classique car le reservoir sampling pondéré n'est pas disponible dans la librairie datafu version 1.2.0.

In [27]:
%%PIG sample_explore_datafu_dist.pig

REGISTER '$CONTAINER/$PSEUDO/sampling/datafu-1.2.0.jar';
DEFINE RS datafu.pig.sampling.ReservoirSample('1000');
DEFINE WeightedSample datafu.pig.sampling.WeightedSample();

ensemble = LOAD '$CONTAINER/$PSEUDO/sampling/sample4.txt' 
        USING PigStorage('\t') AS (x:int, v:chararray) ;
DESCRIBE ensemble;
keys = FOREACH ensemble GENERATE x, v, x%10 AS key;
DESCRIBE keys;
ens_group = GROUP keys BY key ;
DESCRIBE ens_group;
sampled = FOREACH ens_group GENERATE COUNT(keys) AS weigth, FLATTEN(RS(keys));
DESCRIBE sampled;
wsampled = FOREACH (GROUP sampled ALL) GENERATE FLATTEN(WeightedSample(sampled, 0, 1000));
DESCRIBE wsampled;

STORE wsampled 
INTO '$CONTAINER/$PSEUDO/sampling/sample_datafu_rs_dist2.txt' USING PigStorage();

In [28]:
jid = %hd_pig_submit sample_explore_datafu_dist.pig
jid

{'id': 'job_1446540516812_0238'}

In [29]:
st = %hd_job_status jid["id"]
(st["id"],st["percentComplete"],st["completed"],
st["status"]["jobComplete"],st["status"]["state"])

('job_1446540516812_0238', '100% complete', 'done', True, 'SUCCEEDED')

In [30]:
%hd_tail_stderr jid["id"] -n 10

In [31]:
%blob_ls /$PSEUDO/sampling/sample_datafu_rs_dist2

,name,last_modified,content_type,content_length,blob_type
0,axavier/sampling/sample_datafu_rs_dist2.txt,"Sun, 15 Nov 2015 19:22:05 GMT",,0,BlockBlob
1,axavier/sampling/sample_datafu_rs_dist2.txt/_S...,"Sun, 15 Nov 2015 19:22:06 GMT",application/octet-stream,0,BlockBlob
2,axavier/sampling/sample_datafu_rs_dist2.txt/pa...,"Sun, 15 Nov 2015 19:22:05 GMT",application/octet-stream,20770,BlockBlob


In [32]:
df = %blob_head /$PSEUDO/sampling/sample_datafu_rs_dist2.txt -m

In [33]:
df.head()

,10001,21260,S21260,0
0,10000,25191,X25191,1
1,10000,73760,Y73760,0
2,10000,90105,P90105,5
3,10000,46070,Y46070,0
4,10001,58590,M58590,0


## version distribuée améliorée

Le problème de la version précédente : chaque sous-ensemble traité d'un seul bloc utilise une séquence de nombres aléatoires sur laquelle on ne connaît pas grand chose. Si les mêmes *seed* sont utilisées, il est possible que les séquences, même si elles simulent le hasard, soient extrêmement corrélées entre chaque bloc. Il faut remédier à cela.

Il faut également s'assurer que chaque bloc n'est pas *skewed*.

In [34]:
%%PIG_azure script_rs.pig

REGISTER '$CONTAINER/$PSEUDO/sampling/datafu-1.2.0.jar';
DEFINE MD5 datafu.pig.hash.MD5();
DEFINE RS datafu.pig.sampling.ReservoirSample('1000');
DEFINE WeightedSample datafu.pig.sampling.WeightedSample();

ensemble = LOAD '$CONTAINER/$PSEUDO/sampling/sample4.txt' 
        USING PigStorage('\t') AS (x:int, v:chararray) ;
DESCRIBE ensemble;

ens_group = GROUP ensemble BY (x,v);
DESCRIBE ens_group;

compte_group = FOREACH ens_group 
            GENERATE group.x AS x, 
                     group.v AS v, 
                     COUNT(ensemble) AS nb_ligne ;
DESCRIBE compte_group;

hash_group = FOREACH compte_group 
                GENERATE x, v, nb_ligne,
                        SUBSTRING(MD5(v), 0, 1) AS val;
DESCRIBE hash_group;  

group_hash = GROUP hash_group BY val ;
DESCRIBE group_hash;

rs_parall = FOREACH group_hash GENERATE
                    COUNT(hash_group) AS nb_hash,
                    FLATTEN(RS(hash_group)) ;
DESCRIBE rs_parall;

wsampled = FOREACH (GROUP rs_parall ALL) GENERATE FLATTEN(WeightedSample(rs_parall, 0, 1000));
DESCRIBE wsampled;

STORE wsampled 
INTO '$CONTAINER/$PSEUDO/sampling/sample_distributed_hash.txt' USING PigStorage();

In [35]:
jid=%hd_pig_submit script_rs.pig
jid

{'id': 'job_1446540516812_0244'}

In [36]:
st = %hd_job_status jid["id"]
(st["id"],st["percentComplete"],st["completed"],
st["status"]["jobComplete"],st["status"]["state"])

('job_1446540516812_0244', '100% complete', None, False, 'RUNNING')

In [37]:
%hd_tail_stderr jid["id"] -n 10

In [38]:
%blob_ls /$PSEUDO/sampling/sample_distributed_hash.txt

,name,last_modified,content_type,content_length,blob_type
0,axavier/sampling/sample_distributed_hash.txt,"Sun, 15 Nov 2015 19:51:56 GMT",,0,BlockBlob
1,axavier/sampling/sample_distributed_hash.txt/_...,"Sun, 15 Nov 2015 19:51:56 GMT",application/octet-stream,0,BlockBlob
2,axavier/sampling/sample_distributed_hash.txt/p...,"Sun, 15 Nov 2015 19:51:55 GMT",application/octet-stream,21750,BlockBlob


In [39]:
df =%blob_head /$PSEUDO/sampling/sample_distributed_hash.txt -m
df.head()

,6693,27244,W27244,1,6
0,6749,51104,O51104,1,1
1,6605,91527,H91527,1,6
2,6630,75027,R75027,1,4
3,6789,58148,M58148,1,1
4,6659,71659,D71659,1,5


In [40]:
%blob_downmerge /$PSEUDO/sampling/sample_distributed_hash.txt sample_distributed_hash.txt

'sample_distributed_hash.txt'

In [41]:
%head sample_distributed_hash.txt

## version java

On s'inspire de l'exemple suivant [Sampling](http://datafu.incubator.apache.org/docs/datafu/guide/sampling.html).
 On télécharge [datafu 1.2](http://datafu.incubator.apache.org/docs/datafu/) depuis [Maven](http://mvnrepository.com/artifact/com.linkedin.datafu/datafu/1.2.0). Ce n'est pas la dernière version mais suivre les  instructions pour *builder* datafu (voir [documentation](http://datafu.incubator.apache.org/docs/datafu/1.2.0/)). En particulier, la version pondérée du reservoir sampling n'est pas disponible (voir [history](https://github.com/apache/incubator-datafu/commits/master/datafu-pig/src/main/java/datafu/pig/sampling/WeightedReservoirSample.java), la version 1.2.0 est sorti en décembre 2013).
 
L'implémentation [java](https://github.com/apache/incubator-datafu/blob/master/datafu-pig/src/main/java/datafu/pig/sampling/ReservoirSample.java) n'a pas l'air de résoudre un problème qui peut survenir si la taille de l'échantillon demandée est trop grande. Voir section suivante.

In [43]:
import pyensae
pyensae.download_data("datafu-1.2.0.jar", url="http://central.maven.org/maven2/com/linkedin/datafu/datafu/1.2.0/")

'datafu-1.2.0.jar'

In [44]:
%blob_up datafu-1.2.0.jar /$PSEUDO/sampling/datafu-1.2.0.jar

'$PSEUDO/sampling/datafu-1.2.0.jar'

In [45]:
%blob_ls /$PSEUDO/sampling

,name,last_modified,content_type,content_length,blob_type
0,axavier/sampling/datafu-1.2.0.jar,"Fri, 13 Nov 2015 00:03:49 GMT",application/octet-stream,1600826,BlockBlob
1,axavier/sampling/sample.txt,"Fri, 13 Nov 2015 00:02:50 GMT",application/octet-stream,1377780,BlockBlob
2,axavier/sampling/sample2.txt,"Fri, 13 Nov 2015 00:35:55 GMT",application/octet-stream,1377793,BlockBlob
3,axavier/sampling/sample3.txt,"Fri, 13 Nov 2015 00:39:40 GMT",application/octet-stream,1377793,BlockBlob
4,axavier/sampling/sample4.txt,"Fri, 13 Nov 2015 00:41:49 GMT",application/octet-stream,1377793,BlockBlob


In [46]:
%%PIG_azure sample.pig

REGISTER '$CONTAINER/$PSEUDO/sampling/datafu-1.2.0.jar';

DEFINE RS datafu.pig.sampling.ReservoirSample('1000');

dset = LOAD '$CONTAINER/$PSEUDO/sampling/sample4.txt' 
        USING PigStorage('\t') AS (x:int, v:chararray) ;
sampled = FOREACH (GROUP dset ALL) GENERATE FLATTEN(RS(dset));
STORE sampled INTO '$CONTAINER/$PSEUDO/sampling/out_sampled_rs4_2015.txt' USING PigStorage() ;

In [47]:
jid = %hd_pig_submit sample.pig

In [48]:
st = %hd_job_status jid["id"]
st["id"],st["percentComplete"],st["completed"],st["status"]["jobComplete"],st["status"]["state"]

('job_1446540516812_0136', None, None, False, 'RUNNING')

In [49]:
%hd_tail_stderr jid["id"] -n 10

In [50]:
%blob_ls /$PSEUDO/sampling

,name,last_modified,content_type,content_length,blob_type
0,axavier/sampling/datafu-1.2.0.jar,"Fri, 13 Nov 2015 00:03:49 GMT",application/octet-stream,1600826,BlockBlob
1,axavier/sampling/out_sampled_rs4_2015.txt,"Fri, 13 Nov 2015 01:08:22 GMT",,0,BlockBlob
2,axavier/sampling/out_sampled_rs4_2015.txt/_SUC...,"Fri, 13 Nov 2015 01:08:22 GMT",application/octet-stream,0,BlockBlob
3,axavier/sampling/out_sampled_rs4_2015.txt/part...,"Fri, 13 Nov 2015 01:08:21 GMT",application/octet-stream,12785,BlockBlob
4,axavier/sampling/sample.txt,"Fri, 13 Nov 2015 00:02:50 GMT",application/octet-stream,1377780,BlockBlob
5,axavier/sampling/sample2.txt,"Fri, 13 Nov 2015 00:35:55 GMT",application/octet-stream,1377793,BlockBlob
6,axavier/sampling/sample3.txt,"Fri, 13 Nov 2015 00:39:40 GMT",application/octet-stream,1377793,BlockBlob
7,axavier/sampling/sample4.txt,"Fri, 13 Nov 2015 00:41:49 GMT",application/octet-stream,1377793,BlockBlob
8,axavier/sampling/sampled4_2015.txt,"Fri, 13 Nov 2015 00:50:20 GMT",,0,BlockBlob
9,axavier/sampling/sampled4_2015.txt/_SUCCESS,"Fri, 13 Nov 2015 00:50:20 GMT",application/octet-stream,0,BlockBlob


In [51]:
%blob_downmerge /$PSEUDO/sampling/out_sampled_rs4_2015.txt out_sampled_rs4_2015.txt -o

'out_sampled_rs4_2015.txt'

In [52]:
%head out_sampled_rs4_2015.txt

## fin

In [53]:
%blob_close

True

## version avec itérateur